In [22]:
import folium
import os
import json
import pandas as pd
import base64

#from shapely.geometry import MultiPolygon, shape, mapping

#change dir to the location of the data_bourgogne.json file which contains only locations of Burgundy wine
os.chdir('/Users/Gabriel/Documents/Github/Projet_informatique_ENSAE/jsons')
%run '/Users/Gabriel/Documents/Github/Projet_informatique_ENSAE/cartes/scripts/function_addlayer.py'

In [2]:
# Open the file
with open('data_bourgogne_wtype.json') as json_file:
    data = json.load(json_file)
    
file = open("/Users/Gabriel/Documents/Github/Projet_informatique_ENSAE/cartes/scripts/grandscrus.txt",'r')
grandscrus = file.read().split('\n')

for key in data.keys():
    for gc in grandscrus:
        if data[key]['properties']['appellation'] == gc :
            data[key]['properties']['Grand Cru'] = 1

In [3]:
data_climat = {}
data_climat_rouge = {}
data_climat_blanc = {}

for key in data.keys():
    if data[key]['properties']['climat']!= '' :
        data_climat[key] = data[key]

In [4]:
# Export the names of the climats in an external file. 
os.chdir('/Users/Gabriel/Documents/Github/Projet_informatique_ENSAE/cartes/scripts')
f = open('keywords_to_scrap.txt', 'w') 

# loop through the list of wines
for key in data_climat.keys():
    if data_climat[key]['properties']['climat'] != '':
        # if there is a climat, search the 'appellation + climat'
        name = data_climat[key]['properties']['appellation'] +' '+ data_climat[key]['properties']['climat']
    elif data_climat[key]['properties']['Premier Cru'] + data_climat[key]['properties']['Grand Cru'] != 0:
        # elif Grand Cru or Premier Cru, search for the appellation only
        name = data_climat[key]['properties']['appellation']
        # the name of the image is saved, not the image itself in the json
    data_climat[key]['image'] = name
    f.write(str(name)+'\t')

In [5]:
for key in data_climat.keys():
    if data_climat[key]['properties']['type_vin'] == 'rouge':
        if 'mousseux' not in data_climat[key]['properties']['appellation'].lower() and "Crémant" not in data_climat[key]['properties']['appellation'] :
            data_climat_rouge[key] = data_climat[key]
        
    if data_climat[key]['properties']['type_vin'] == 'blanc':
        if 'mousseux' not in data_climat[key]['properties']['appellation'].lower() and "Crémant" not in data_climat[key]['properties']['appellation'] and data_climat[key]['properties']['climat']!='Côte d\'Or' :
            data_climat_blanc[key] = data_climat[key]
            

In [23]:
style_redwine = {'fillColor': '#DC1A40', 'color': '#464140', 'fill_opacity': 1, 'line_opacity' : 0.5}
style_whitewine = {'fillColor': '#EAE010', 'color': '#464140', 'fill_opacity': 1, 'line_opacity' : 0.5}

os.chdir('/Users/Gabriel/Documents/Github/Projet_informatique_ENSAE')

wine_map = folium.Map(location=[47.2334, 4.96],
                      height=500,
                      tiles = "Stamen Terrain",
                      zoom_start=12,
                      prefer_canvas=True,
                      max_zoom = 15)

# don't forget to change the directories also in the python script.
add_layertomap(data_climat_rouge, style_redwine, 'Climat de Bourgogne Rouge', wine_map)
add_layertomap(data_climat_blanc, style_whitewine, 'Climat de Bourgogne Blanc', wine_map)
folium.LayerControl().add_to(wine_map)

#wine_map.save(os.path.join('results', 'climats.html'))
wine_map.save('/Users/Gabriel/Documents/Github/Projet_informatique_ENSAE/cartes/climats.html')